In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torchvision import transforms

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load and preprocess data
train_data = pd.read_csv("/kaggle/input/facialrecog/train_data.csv", header=None)
train_targets = pd.read_csv("/kaggle/input/facialrecog/train_target.csv", header=None)
test_data = pd.read_csv("/kaggle/input/facialrecog/test_data.csv", header=None)

# Convert to PyTorch tensors
X_train = torch.Tensor(train_data.values.reshape(-1, 1, 48, 48)).to(device)  # Assuming images are grayscale
y_train = torch.LongTensor(train_targets.values.flatten()).to(device)

X_test = torch.Tensor(test_data.values.reshape(-1, 1, 48, 48)).to(device)

# Normalize pixel values to the range [0, 1]
X_train /= 255.0
X_test /= 255.0

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Data Augmentation
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor()
])

augmented_dataset = TensorDataset(X_train, y_train)

# Define a Neural Network
class FacialExpressionModel(nn.Module):
    def __init__(self):
        super(FacialExpressionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 3)  # 3 classes: Angry, Happy, Neutral
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(F.relu(self.batch_norm1(self.conv1(x))))
        x = self.pool(F.relu(self.batch_norm2(self.conv2(x))))
        x = self.pool(F.relu(self.batch_norm3(self.conv3(x))))
        x = x.view(-1, 128 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Loss Function and Optimizer
model = FacialExpressionModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training Loop
num_epochs = 15
batch_size = 64

train_loader = DataLoader(dataset=augmented_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        # Move data to GPU
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    # Learning rate schedule step
    scheduler.step()

# Validation
model.eval()
with torch.no_grad():
    val_outputs = model(X_val)
    val_loss = criterion(val_outputs, y_val)
    _, val_preds = torch.max(val_outputs, 1)

    accuracy = (val_preds == y_val).sum().item() / len(y_val)
    print(f'Validation Accuracy: {accuracy:.4f}')

# Test the Model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, test_preds = torch.max(test_outputs, 1)

# Save Predictions to CSV
submission_df = pd.DataFrame({'Id': range(len(test_preds)), 'Category': test_preds.cpu().numpy()})
submission_df.to_csv('/kaggle/working/predictions.csv', index=False)


Using device: cpu
Epoch [1/10], Step [100/203], Loss: 1.0179
Epoch [1/10], Step [200/203], Loss: 0.8406
Epoch [2/10], Step [100/203], Loss: 0.8278
Epoch [2/10], Step [200/203], Loss: 0.6997
Epoch [3/10], Step [100/203], Loss: 0.6961
Epoch [3/10], Step [200/203], Loss: 0.6798
Epoch [4/10], Step [100/203], Loss: 0.4683
Epoch [4/10], Step [200/203], Loss: 0.4783
Epoch [5/10], Step [100/203], Loss: 0.5607
Epoch [5/10], Step [200/203], Loss: 0.6677
Epoch [6/10], Step [100/203], Loss: 0.5872
Epoch [6/10], Step [200/203], Loss: 0.6245
Epoch [7/10], Step [100/203], Loss: 0.4538
Epoch [7/10], Step [200/203], Loss: 0.4978
Epoch [8/10], Step [100/203], Loss: 0.4469
Epoch [8/10], Step [200/203], Loss: 0.3739
Epoch [9/10], Step [100/203], Loss: 0.2991
Epoch [9/10], Step [200/203], Loss: 0.4666
Epoch [10/10], Step [100/203], Loss: 0.3619
Epoch [10/10], Step [200/203], Loss: 0.4448
Validation Accuracy: 0.7651


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session